In [1]:
from pathlib import Path
import torch
import matplotlib.pyplot as plt
import numpy as np
import random
from torch.utils import data
from torch import nn
%matplotlib inline

In [2]:
def sythentic_data(w, b, rows):
    with torch.no_grad():
        x = torch.normal(0, 1, size=(rows, len(w))) # normal函数中的size参数必须为元组类型
        y = x @ w + b + torch.normal(0, 0.01, size=(rows,))
    return x, y

In [3]:
def random_data(x, y, batch):
    index = list(range(len(x)))
    random.shuffle(index)
    for i in range(0, len(y), batch):
        id = index[i:min(i+batch, len(y))]
        yield x[id], y[id]

In [4]:
def lossFunction(x, y, w, b, func):
    loss = torch.pow(func(x, w, b) - y, 2)
    return torch.sum(loss) / (2 * len(y))

In [5]:
def optimize(*params, alpha):
    with torch.no_grad():
        for param in params:
            param.data -= alpha * param.grad
            param.grad.zero_()

In [6]:
rows = 200
w = torch.normal(0, 1, size=(2,), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
w_true = torch.tensor([0.5, 1.2])
b_true = torch.tensor(2.)
alpha = 0.03
epoch = 20
batch = 20
x, y = sythentic_data(w_true, b_true, rows)
func = lambda x, w, b :x @ w + b

In [7]:
plt.rcParams["font.sans-serif"] = "SimHei"
plt.rcParams["axes.unicode_minus"] = False

In [8]:
plt.close(1)
plt.figure(1)
plt.scatter(x.numpy()[:, 0], y.numpy())

In [ ]:
J = np.empty(shape=0)
for _ in range(epoch):
    l = np.empty(shape=0)
    for bx, by in random_data(x, y, batch):
        loss = lossFunction(bx, by, w, b, func)
        l = np.append(l, loss.data.numpy())
        loss.backward()
        optimize(w, b, alpha=alpha)
    J = np.append(J, l.mean())
del l

In [ ]:
plt.close(2)
plt.figure(2)
plt.plot(range(len(J)), J)

In [ ]:
# 错误总结
'''x = torch.normal(0, 1, size=(rows, len(w)))
y = x @ w + b + torch.normal(0, 0.01, size=(rows,)) # 没有关闭数据构造函数的梯度，程序自动对y求导，导致梯度为0
loss = torch.pow((x @ w + b - y), 2)
loss.sum().backward()
w.grad'''

Error: Session cannot generate requests

## 交叉熵损失函数
$$
loss = -\sum_{i=0}^{n} y_i \log \frac{e^{o_i}}{\sum_{j=0}^{m} e^{o_j}}
$$

In [ ]:
x1, y1 = sythentic_data(w_true, b_true, rows)
temp = data.TensorDataset(x1, y1)
dataset = data.DataLoader(temp, batch, shuffle=True)
net = nn.Sequential(nn.Linear(2, 1))
mseloss = nn.MSELoss()
net[0].weight.data.normal_(0.1, 0.01)
net[0].bias.data.fill_(0)
trainer = torch.optim.SGD(net.parameters(), lr=0.03)
for _ in range(epoch):
    for xx, yy in dataset:
        loss = mseloss(net(xx), yy.reshape(-1, 1))
        trainer.zero_grad()
        loss.backward()
        trainer.step()

In [ ]:
net[0].weight.data, net[0].bias.data